In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_profiling import ProfileReport
plt.rcParams['figure.figsize'] = (16,10)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("/kaggle/input/a-fine-windy-day-hackerearth-ml-challenge/train_data.csv")
train.head()

In [ ]:
test = pd.read_csv("/kaggle/input/a-fine-windy-day-hackerearth-ml-challenge/test_data.csv")
test.head()

In [ ]:
list1 = train.columns
list2 = test.columns
print(set(list1).difference(list2))

In [ ]:
train.info(verbose = True)


In [ ]:
train.describe()

In [ ]:
test.describe()

In [ ]:
train["windmill_generated_power(kW/h)"].describe()

In [ ]:
train.nunique()

In [ ]:
test.nunique()

In [ ]:
sns.heatmap(train.isnull(),cbar=False,yticklabels=False,cmap = 'viridis')

In [ ]:
sns.heatmap(test.isnull(),cbar = False, yticklabels = False, cmap = 'viridis')

In [ ]:
train.isna().sum()

In [ ]:
corr = train.corr()
plt.figure(figsize=(20,10))
mask = np.zeros_like(corr,dtype=np.bool)
#print(mask)
mask[np.triu_indices_from(mask)] = True
sns.heatmap(corr,mask=mask,annot=True)
plt.show()

In [ ]:
def getFeatureType(dataframe):
    num_features = dataframe.select_dtypes(include=[np.number])
    cat_features = dataframe.select_dtypes(include=[np.object])
    return dataframe, cat_features

In [ ]:
num_feature, cat_features = getFeatureType(train)

In [ ]:
#num_feature = num_feature.drop("tracking_id", axis =1)
num_feature = train.drop(["tracking_id", "datetime", "turbine_status", "cloud_level"], axis =1)
num_feature


In [ ]:
cat_features

In [ ]:
print(cat_features["turbine_status"].unique())
print(cat_features["turbine_status"].nunique())

In [ ]:
print(cat_features["cloud_level"].unique())
print(cat_features["cloud_level"].nunique())

In [ ]:
#print(cat_features["tracking_id"].nunique())
#print(cat_features["datetime"].nunique())

In [ ]:
#Box plot
# fig = plt.figure(figsize = (30,20))
# for i in range(len(num_feature.columns)):
#     fig.add_subplot(4,5, i+1)
#     sns.boxplot(y = num_feature.iloc[:,i])
# plt.tight_layout()
# plt.show()

# fig = plt.figure(figsize=(30,20))
# for i in range(len(num_feature.columns)):
#     fig.add_subplot(4,5,1)
#     sns.boxplot(num_feature.iloc[:,i])
# plt.tight_layout()
# plt.show()

# sns.boxplot(num_feature.iloc[:,1])

In [ ]:
sns.pairplot(train)
plt.show()

In [ ]:
skew_features = num_feature.apply(lambda x :x.skew()).sort_values(ascending=True)
skew_features

In [ ]:
train_copy = train.copy()

In [ ]:
def comparing_train_and_test_feature(df,df_test,col):
    fig = plt.figure(figsize=(16,10))
    ax0 = fig.add_subplot(1,2,1)
    ax1 = fig.add_subplot(1,2,2)
    df[col].plot(kind='kde',ax=ax0)
    df_test[col].plot(kind='kde',ax=ax1)
    ax0.set_xlabel(col)
    ax1.set_xlabel(col)
    ax0.set_title("Density plot of " + str(col) + " of training set")
    ax1.set_title("Density plot of " + str(col) + " of testing set")
    plt.show()

In [ ]:
comparing_train_and_test_feature(train,test,'wind_speed(m/s)')

In [ ]:
sns.boxplot(y='wind_speed(m/s)',data=train)

In [ ]:
sns.scatterplot(x='wind_speed(m/s)',y='windmill_generated_power(kW/h)',hue='cloud_level',data=train)

In [ ]:
train['wind_speed(m/s)'].value_counts()

In [ ]:
comparing_train_and_test_feature(train,test,'atmospheric_temperature(°C)')

In [ ]:
sns.boxplot(y='atmospheric_temperature(°C)',data=train)

In [ ]:
sns.scatterplot(x='atmospheric_temperature(°C)',y='windmill_generated_power(kW/h)',hue='cloud_level',data=train)

In [ ]:
train[train['atmospheric_temperature(°C)'] < -50]

In [ ]:
comparing_train_and_test_feature(train,test,'shaft_temperature(°C)')

In [ ]:
sns.boxplot(y='shaft_temperature(°C)',data=train)

In [ ]:
sns.scatterplot(x='shaft_temperature(°C)',y='windmill_generated_power(kW/h)',hue='cloud_level',data=train)

In [ ]:
plt.figure(figsize=(20,10))
sns.scatterplot(x='shaft_temperature(°C)',y='wind_speed(m/s)',hue='cloud_level',data=train)

In [ ]:
comparing_train_and_test_feature(train,test,'blades_angle(°)')

In [ ]:
sns.boxplot(y='blades_angle(°)',data=train)

In [ ]:
sns.scatterplot(x='blades_angle(°)',y='windmill_generated_power(kW/h)',hue='cloud_level',data=train)

In [ ]:
comparing_train_and_test_feature(train,test,'gearbox_temperature(°C)')

In [ ]:
sns.boxplot(y='gearbox_temperature(°C)',data=train)

In [ ]:
sns.scatterplot(x='gearbox_temperature(°C)',y='windmill_generated_power(kW/h)',hue='cloud_level',data=train)

In [ ]:
low = train_copy['gearbox_temperature(°C)'] < -200
high = train_copy['gearbox_temperature(°C)'] > 300
low = np.where(low)
high = np.where(high)
train_copy.drop(low[0],inplace=True)
train_copy.drop(high[0],inplace=True)
train_copy.index = range(train_copy.shape[0])

In [ ]:
plt.figure(figsize=(20,10))
sns.scatterplot(x='gearbox_temperature(°C)',y='windmill_generated_power(kW/h)',hue='cloud_level',data=train_copy)

In [ ]:
train_copy[train_copy['gearbox_temperature(°C)']<-200]

In [ ]:
train_copy[train_copy['gearbox_temperature(°C)'] > 300]

In [ ]:
comparing_train_and_test_feature(train,test,'engine_temperature(°C)')

In [ ]:
sns.boxplot(y='engine_temperature(°C)',data=train)

In [ ]:
sns.scatterplot(x='engine_temperature(°C)',y='windmill_generated_power(kW/h)',hue='cloud_level',data=train)

In [ ]:
low = train_copy['engine_temperature(°C)'] < 38
low = np.where(low)
train_copy.drop(low[0],inplace=True)
train_copy.index = range(train_copy.shape[0])

In [ ]:
plt.figure(figsize=(20,10))
sns.scatterplot(x='engine_temperature(°C)',y='windmill_generated_power(kW/h)',hue='cloud_level',data=train_copy)

In [ ]:
comparing_train_and_test_feature(train,test,'motor_torque(N-m)')

In [ ]:
sns.boxplot(y='motor_torque(N-m)',data=train)

In [ ]:
sns.scatterplot(x='motor_torque(N-m)',y='windmill_generated_power(kW/h)',hue='cloud_level',data=train)

In [ ]:
comparing_train_and_test_feature(train,test,'generator_temperature(°C)')

In [ ]:
sns.boxplot(y='generator_temperature(°C)',data=train)

In [ ]:
sns.scatterplot(x='generator_temperature(°C)',y='windmill_generated_power(kW/h)',hue='cloud_level',data=train)

In [ ]:
comparing_train_and_test_feature(train,test,'atmospheric_pressure(Pascal)')

In [ ]:
sns.boxplot(y='atmospheric_pressure(Pascal)',data=train)

In [ ]:
sns.scatterplot(x='atmospheric_pressure(Pascal)',y='windmill_generated_power(kW/h)',hue='cloud_level',data=train)

In [ ]:
comparing_train_and_test_feature(train,test,'area_temperature(°C)')

In [ ]:
sns.boxplot(y='area_temperature(°C)',data=train)

In [ ]:
sns.scatterplot(x='area_temperature(°C)',y='windmill_generated_power(kW/h)',hue='cloud_level',data=train)

In [ ]:
comparing_train_and_test_feature(train,test,'windmill_body_temperature(°C)')

In [ ]:
sns.boxplot(y='windmill_body_temperature(°C)',data=train)

In [ ]:
sns.scatterplot(x='windmill_body_temperature(°C)',y='windmill_generated_power(kW/h)',hue='cloud_level',data=train)

In [ ]:
train[train['windmill_body_temperature(°C)']< -90]

In [ ]:
comparing_train_and_test_feature(train,test,'wind_direction(°)')

In [ ]:
sns.boxplot(y='wind_direction(°)',data=train)

In [ ]:
sns.scatterplot(x='wind_direction(°)',y='windmill_generated_power(kW/h)',hue='cloud_level',data=train)

In [ ]:
comparing_train_and_test_feature(train,test,'resistance(ohm)')

In [ ]:
sns.boxplot(y='resistance(ohm)',data=train)

In [ ]:
sns.scatterplot(x='resistance(ohm)',y='windmill_generated_power(kW/h)',hue='cloud_level',data=train)

In [ ]:
comparing_train_and_test_feature(train,test,'rotor_torque(N-m)')

In [ ]:
sns.boxplot(y='rotor_torque(N-m)',data=train)

In [ ]:
sns.scatterplot(x='rotor_torque(N-m)',y='windmill_generated_power(kW/h)',hue='cloud_level',data=train)

In [ ]:
comparing_train_and_test_feature(train,test,'blade_length(m)')

In [ ]:
sns.boxplot(y='blade_length(m)',data=train)

In [ ]:
sns.scatterplot(x='blade_length(m)',y='windmill_generated_power(kW/h)',hue='cloud_level',data=train)

In [ ]:
low = train_copy['blade_length(m)'] < -20
low = np.where(low)
train_copy.drop(low[0],inplace=True)
train_copy.index = range(train_copy.shape[0])

In [ ]:
plt.figure(figsize=(20,10))
sns.scatterplot(x='blade_length(m)',y='windmill_generated_power(kW/h)',hue='cloud_level',data=train_copy)

In [ ]:
comparing_train_and_test_feature(train,test,'blade_breadth(m)')

In [ ]:
sns.boxplot(y='blade_breadth(m)',data=train)

In [ ]:
sns.scatterplot(x='blade_breadth(m)',y='windmill_generated_power(kW/h)',hue='cloud_level',data=train)

In [ ]:
comparing_train_and_test_feature(train,test,'windmill_height(m)')

In [ ]:
sns.boxplot(y='windmill_height(m)',data=train)

In [ ]:
sns.scatterplot(x='windmill_height(m)',y='windmill_generated_power(kW/h)',hue='cloud_level',data=train)

In [ ]:
train_copy.drop(['generator_temperature(°C)','windmill_body_temperature(°C)'],inplace=True,axis=1)
test.drop(['generator_temperature(°C)','windmill_body_temperature(°C)'],inplace=True,axis=1)

In [ ]:
train_copy.info()

In [ ]:
train_copy.describe()

In [ ]:
train_copy['area_temperature(°C)'].fillna(train_copy['area_temperature(°C)'].mean(),inplace=True)
train_copy['rotor_torque(N-m)'].fillna(train_copy['rotor_torque(N-m)'].mean(),inplace=True)
train_copy['blade_length(m)'].fillna(train_copy['blade_length(m)'].mean(),inplace=True)
train_copy['blade_breadth(m)'].fillna(train_copy['blade_breadth(m)'].mean(),inplace=True)
train_copy['windmill_height(m)'].fillna(train_copy['windmill_height(m)'].mean(),inplace=True)
train_copy['cloud_level'].fillna(train_copy['cloud_level'].mode()[0],inplace=True)
train_copy['atmospheric_temperature(°C)'].fillna(train_copy['atmospheric_temperature(°C)'].mean(),inplace=True)
train_copy['atmospheric_pressure(Pascal)'].fillna(train_copy['atmospheric_pressure(Pascal)'].mean(),inplace=True)
train_copy['wind_speed(m/s)'].fillna(train_copy['wind_speed(m/s)'].mean(),inplace=True)
train_copy['shaft_temperature(°C)'].fillna(train_copy['shaft_temperature(°C)'].mean(),inplace=True)
train_copy['blades_angle(°)'].fillna(train_copy['blades_angle(°)'].mean(),inplace=True)
train_copy['engine_temperature(°C)'].fillna(train_copy['engine_temperature(°C)'].mean(),inplace=True)
train_copy['motor_torque(N-m)'].fillna(train_copy['motor_torque(N-m)'].mean(),inplace=True)
train_copy['wind_direction(°)'].fillna(train_copy['wind_direction(°)'].mean(),inplace=True)

In [ ]:
test['gearbox_temperature(°C)'].fillna(test['gearbox_temperature(°C)'].mean(),inplace=True)
test['area_temperature(°C)'].fillna(test['area_temperature(°C)'].mean(),inplace=True)
test['rotor_torque(N-m)'].fillna(test['rotor_torque(N-m)'].mean(),inplace=True)
test['blade_length(m)'].fillna(test['blade_length(m)'].mean(),inplace=True)
test['blade_breadth(m)'].fillna(test['blade_breadth(m)'].mean(),inplace=True)
test['windmill_height(m)'].fillna(test['windmill_height(m)'].mean(),inplace=True)
test['cloud_level'].fillna(test['cloud_level'].mode()[0],inplace=True)
test['atmospheric_temperature(°C)'].fillna(test['atmospheric_temperature(°C)'].mean(),inplace=True)
test['atmospheric_pressure(Pascal)'].fillna(test['atmospheric_pressure(Pascal)'].mean(),inplace=True)
test['wind_speed(m/s)'].fillna(test['wind_speed(m/s)'].mean(),inplace=True)
test['shaft_temperature(°C)'].fillna(test['shaft_temperature(°C)'].mean(),inplace=True)
test['blades_angle(°)'].fillna(test['blades_angle(°)'].mean(),inplace=True)
test['engine_temperature(°C)'].fillna(test['engine_temperature(°C)'].mean(),inplace=True)
test['motor_torque(N-m)'].fillna(test['motor_torque(N-m)'].mean(),inplace=True)
test['wind_direction(°)'].fillna(test['wind_direction(°)'].mean(),inplace=True)


In [ ]:
train_copy.info()

In [ ]:
train_copy.dropna(how='any',axis=0,inplace=True)

In [ ]:
train_copy.info()

In [ ]:
test.info()

In [ ]:
#train_copy["cloud_level"].unique()

In [ ]:
train_copy["cloud_level"].replace(["Medium","Low","Extremely Low"], [2,1,0], inplace = True)
test["cloud_level"].replace(["Medium","Low","Extremely Low"], [2,1,0], inplace = True)


In [ ]:
#train_copy['turbine_status'].value_counts()
pd.options.display.max_columns = None
print(list(train_copy.columns))
#train_copy.head(3)
test.head(3)


In [ ]:
dum = ['turbine_status']
df_dum = pd.get_dummies(train_copy[dum])
df_test_dum = pd.get_dummies(test[dum])
df_dum

In [ ]:
train_copy = pd.concat([train_copy,df_dum],axis=1)
test = pd.concat([test,df_test_dum],axis=1)

In [ ]:
print(list(train_copy.columns))
#print(list(test.columns))


In [ ]:
train_copy['datetime'] = pd.to_datetime(train_copy['datetime'])
test['datetime'] = pd.to_datetime(test['datetime'])

In [ ]:
train_copy['dmonth'] = train_copy['datetime'].dt.month
train_copy['dday'] = train_copy['datetime'].dt.day
train_copy['ddayofweek'] = train_copy['datetime'].dt.dayofweek

test['dmonth'] = test['datetime'].dt.month
test['dday'] = test['datetime'].dt.day
test['ddayofweek'] = test['datetime'].dt.dayofweek

In [ ]:
#train_copy['dmonth']
# train_copy['ddayofweek']
# train_copy['dday']

In [ ]:
corr = train_copy.corr()
plt.figure(figsize=(30,15))
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
sns.heatmap(corr, mask=mask, annot = True)
plt.show()

In [ ]:
corr = test.corr()
plt.figure(figsize=(30,10))
mask = np.zeros_like(corr, dtype = np.bool)
mask[np.triu_indices_from(mask)] = True
sns.heatmap(corr, mask= mask, annot =True)
plt.show()



In [ ]:
X = train_copy.drop(['tracking_id','datetime','windmill_generated_power(kW/h)','turbine_status'],axis=1)
y = train_copy["windmill_generated_power(kW/h)"]
X_test = test.drop(['tracking_id','datetime','turbine_status'], axis =1)

# (X.shape, y.shape)
# print(X_test.shape)
# #print(X.head())
# print(X_test.head())



In [ ]:
#X.head(3)
#X_test.head(3)

In [ ]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
X= scaler.fit_transform(X)
X_test= scaler.fit_transform(X_test)
#df1 = X_test["cloud_level"].str.contains("Extremely Low")
#X_test.head()



In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)
print(x_train.shape, y_train.shape)
print(x_test.shape,y_test.shape)

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import r2_score
lr = LinearRegression()
lr.fit(x_train, y_train)
y_train_predict = lr.predict(x_train)
y_test_predict = lr.predict(x_test)
print(r2_score(y_true = y_train, y_pred = y_train_predict))
print(r2_score(y_true=y_test, y_pred = y_test_predict))




In [ ]:
from sklearn.linear_model import LassoCV
from sklearn.metrics import r2_score
lasso_model = LassoCV()
lasso_model.fit(x_train, y_train)
y_train_predict = lasso_model(x_train)
y_test_predict = lasso_model(x_test)
print(r2_score(y_true = y_train, y_pred = y_train_predict))
print(r2_score(y_true = y_test, y_pred = y_test_predict))

In [ ]:
from sklearn.linear_model import ElasticNetCV
enet_model = ElasticNetCV(l1_ratio = [0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1],
                    alphas = [1, 0.1, 0.01, 0.001, 0.0005], cv=5)
enet_model.fit(x_train, y_train)

# predict
y_train_pred = enet_model.predict(x_train)
y_test_pred = enet_model.predict(x_test)
print(r2_score(y_train,y_train_pred))
print(r2_score(y_test,y_test_pred))

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
extra_model = ExtraTreesRegressor(criterion='mse', random_state=0, n_jobs=-1, min_samples_leaf=1, max_depth=20, min_samples_split=3, n_estimators=1000)
extra_model.fit(x_train, y_train)
y_train_pred = extra_model.predict(x_train)
y_test_pred = extra_model.predict(x_test)
print(r2_score(y_train,y_train_pred))
print(r2_score(y_test,y_test_pred))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf= RandomForestRegressor()
rf.fit(x_train, y_train)
y_train_predict = rf.predict(x_train)
y_test_predict = rf.predict(x_test)
print(r2_score(y_train, y_train_predict))
print(r2_score(y_test, y_test_predict))

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
gb_model= GradientBoostingRegressor(criterion='mse', random_state=0, max_depth=5, n_estimators=500, min_samples_split=2, min_samples_leaf=2)
gb_model.fit(x_train, y_train)
y_train_predict= gb_model.predict(x_train)
y_test_predict = gb_model.predict(x_test)
print(r2_score(y_train, y_train_predict))
print(r2_score(y_test, y_test_predict))

In [ ]:
from xgboost import XGBRegressor
xgb = XGBRegressor(n_estimators=500, max_depth=5, booster='gbtree', n_jobs=-1, learning_rate=0.1, reg_lambda=0.01, reg_alpha=0.3)
xgb.fit(x_train, y_train)
y_train_predict= xgb.predict(x_train)
y_test_predict = xgb.predict(x_test)
print(r2_score(y_train, y_train_predict))
print(r2_score(y_test, y_test_predict))


In [ ]:
df_sub =test[['tracking_id','datetime']]

In [ ]:
results = xgb.predict(X_test)
#results

In [ ]:
df_sub['windmill_generated_power(kW/h)'] = results
df_sub

In [ ]:
df_sub.to_csv('power_output.csv', header= True, index= False)
df_sub